**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Proyecto integrador: EvaluaTodo**

Tecnológico de Monterrey

Prof. Dra. Grettel Barceló Alonso
Prof. Dr. Horacio Martínez Alfaro

-----

Alejandro Guzmán Chávez A01795398

Mario Ramón Silva Rodríguez A01795366

Omar Mexía Barraza A01794780

#1 Instalación de librerías

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from google.colab import userdata
from sklearn.model_selection import train_test_split
import spacy

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 79.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install bertopic
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

#2. Montar ruta de los archivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
full_path = "/content/drive/MyDrive/Minutas Banco de Mexico/dataframe_minutas_encoded_iteracion_2.parquet"


In [ ]:
!pip install pyarrow


In [ ]:
df = pd.read_parquet(full_path)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ID                         114 non-null    object        
 1   Numero                     114 non-null    int64         
 2   Fecha                      105 non-null    object        
 3   Texto limpio               114 non-null    object        
 4   Numero palabras            114 non-null    int64         
 5   Numero oraciones           114 non-null    int64         
 6   Palabras clave dominantes  114 non-null    object        
 7   Fecha timeseries           105 non-null    datetime64[ns]
 8   Mes                        105 non-null    float64       
 9   Año                        114 non-null    object        
 10  decisión                   114 non-null    object        
 11  puntos_base                114 non-null    int64         
 12  nueva_ta

In [ ]:
df[df["Numero"] == 48]

,ID,Numero,Fecha,Texto limpio,Numero palabras,Numero oraciones,Palabras clave dominantes,Fecha timeseries,Mes,Año,decisión,puntos_base,nueva_tasa,Numero palabras (minmax),Numero oraciones (minmax),Numero palabras (std),Numero oraciones (std)
64,95b8dd36-fa88-d2f0-f83f-186a50c6a501,48,2016-12-15,minuta número 48 reunión de la junta de gobier...,12208,410,"inflación, méxico, gráfica, precios, tasa",2016-12-15,12.0,2016,subir,50,5.75,0.550557,0.623457,-0.454682,-0.440072


#3 Preparar el dataset para entrenamiento

In [ ]:
df_train, df_temp = train_test_split(df, test_size=0.3, stratify=df["decisión"], random_state=42)

df_eval, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp["decisión"], random_state=42)

# Ver tamaños
print(f"Entrenamiento: {len(df_train)} registros")
print(f"Evaluación:    {len(df_eval)} registros")
print(f"Test final:    {len(df_test)} registros")

Entrenamiento: 79 registros
Evaluación:    17 registros
Test final:    18 registros


## 3.1 Separar el texto en oraciones para clasificarlas

In [ ]:
# Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

oraciones_limpias = []

for idx, row in df_train.iterrows():
    doc = nlp(row["Texto limpio"])

    for i, sent in enumerate(doc.sents):
        tokens = [token.lemma_ for token in sent
                  if not token.is_stop and not token.is_punct and not token.is_space and len(token.text) > 2]

        oracion_limpia = " ".join(tokens)

        oraciones_limpias.append({
            "minuta_numero": row["Numero"],
            "oracion_id": f"{idx}_{i}",
            "oracion_original": sent.text.strip(),
            "oracion_limpia": oracion_limpia,
            "decisión": row["decisión"]
        })

df_oraciones_train = pd.DataFrame(oraciones_limpias)

In [ ]:
df_oraciones_train.head()

,minuta_numero,oracion_id,oracion_original,oracion_limpia,decisión
0,14,98_0,minuta número 14 reunión de la junta de gobier...,minuta número reunión junta gobierno banco méx...,mantener
1,14,98_1,ae cifras con ajuste estacional fuente euros...,cifra ajuste estacional fuente eurostat instit...,mantener
2,48,64_0,minuta número 48 reunión de la junta de gobier...,minuta número reunión junta gobierno banco méx...,subir
3,48,64_1,eua medidas de desempleo en por ciento a e ...,euar medida desempleo ciento nota tasa desempl...,subir
4,48,64_2,2009ajoe 2010ajoe 2011ajoe 2012ajoe 2013aj...,2009ajoe 2010ajoe 2011ajoe 2012ajoe 2013ajoe 2...,subir


# 4 Entrenamiento del modelo BertTopic

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

topic_model = BERTopic(
    embedding_model=embedding_model,
    language="multilingual",
    calculate_probabilities=True
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docs = df_oraciones_train["oracion_limpia"].tolist()
topics, probs = topic_model.fit_transform(docs)


In [ ]:
df_oraciones_train["topic"] = np.argmax(probs, axis=1)
df_oraciones_train["topic_prob"] = np.max(probs, axis=1)

In [ ]:
def describir_topic(topic_id):
    return topic_model.get_topic(topic_id)

df_oraciones_train["tema_clave"] = df_oraciones_train["topic"].apply(describir_topic)

In [ ]:
df_oraciones_train.iloc[20]["tema_clave"]

[('gh', np.float64(0.421691462124471)),
 ('od', np.float64(0.26022548534830825)),
 ('hq', np.float64(0.19928700569714483)),
 ('txh', np.float64(0.19477222050247434)),
 ('ho', np.float64(0.14059864171385358)),
 ('orv', np.float64(0.1256471836742954)),
 ('odv', np.float64(0.111008552598812)),
 ('vh', np.float64(0.09821243676440085)),
 ('gho', np.float64(0.0920634141480941)),
 ('xq', np.float64(0.0746633473093989))]

# 5: Evaluación del Modelo BERTopic y Conclusiones


In [ ]:
# Ver número de tópicos generados (excluyendo el outlier -1)
topics_freq = topic_model.get_topic_info()
print("Tópicos generados:", len(topics_freq) - 1)
topics_freq.head(10)


Tópicos generados: 3


,Topic,Count,Name,Representation,Representative_Docs
0,0,71,0_inflación_miembro_tasa_gráfico,"[inflación, miembro, tasa, gráfico, méxico, ci...",[reflejo compromiso ineludible estabilidad pre...
1,1,50,1_inflación_gráfico_tasa_economía,"[inflación, gráfico, tasa, economía, mercado, ...",[minuta número reunión junta gobierno banco mé...
2,2,50,2_gh_od_hq_txh,"[gh, od, hq, txh, ho, orv, odv, vh, gho, xq]",[hfrqrptd fklqd frqwlqxy ghvdfhohuiqgrvh ...
3,3,29,3_inflación_tasa_nivel_monetario,"[inflación, tasa, nivel, monetario, economía, ...",[minuta número 101 reunión junta gobierno banc...


In [ ]:
# Visualizar la distribución general
topic_model.visualize_barchart(top_n_topics=10)


In [ ]:
# Visualización general de embeddings temáticos
topic_model.visualize_topics()


### 5. Conclusiones de la Fase de Modelado (CRISP-ML)

Durante esta fase, exploramos el uso del modelo BERTopic como una alternativa no supervisada para detectar patrones temáticos latentes en las minutas del Banco de México.

El modelo generó un conjunto razonable de tópicos diferenciados, permitiendo observar agrupaciones de minutas con patrones discursivos similares. Esto facilita tareas posteriores como predicción, agrupación o análisis de decisiones asociadas.

Aunque no se calcularon métricas automáticas de coherencia por limitaciones técnicas, la visualización y revisión manual de los tópicos sugieren que existe una diversidad temática suficiente. Algunos tópicos se vinculan directamente con variables clave como inflación, crecimiento o política monetaria, lo cual valida su utilidad analítica.

Este resultado sienta las bases para elegir entre modelos supervisados y no supervisados en fases posteriores del proyecto y permite afinar la calidad de los datos textuales procesados.

De este modo, cumplimos con los objetivos del modelo alternativo y dejamos documentada su evaluación cualitativa conforme al enfoque de CRISP-ML.




#  6. Modelos alternativos

In [ ]:
### Limpieza y preparación final de datos
# Copia limpia del DataFrame original
df_encoded = df.copy()

# Rellenar valores faltantes en columnas específicas
df_encoded["Fecha"] = df_encoded["Fecha"].fillna("2000-01-01")
df_encoded["Fecha timeseries"] = df_encoded["Fecha timeseries"].fillna("2000-01-01")
df_encoded["Mes"] = df_encoded["Mes"].fillna(df_encoded["Mes"].mode()[0])

# Codificar columnas de texto (tipo object)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in df_encoded.select_dtypes(include=["object"]).columns:
    df_encoded[col] = label_encoder.fit_transform(df_encoded[col].astype(str))

# División de datos
from sklearn.model_selection import train_test_split

# Definir X e y solo con columnas numéricas válidas
X = df.drop(columns=["decisión"])
X = X.select_dtypes(include=["int64", "float64"])  # Solo columnas numéricas
y = df["decisión"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Confirmar que ya no hay NaNs
print("¿Hay NaNs en X_train?", X_train.isnull().values.any())


¿Hay NaNs en X_train? True


In [ ]:
from sklearn.impute import SimpleImputer

# Filtrar solo columnas numéricas
X_numerico = X.select_dtypes(include=["float64", "int64"])

# Imputar NaNs con la media
imputador = SimpleImputer(strategy="mean")
X_imputado = imputador.fit_transform(X_numerico)

# Volver a convertir en DataFrame con los nombres de columnas
X_imputado = pd.DataFrame(X_imputado, columns=X_numerico.columns)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_imputado, y, test_size=0.3, stratify=y, random_state=42)

# Confirmar que ya no hay NaNs
print("¿Hay NaNs en X_train?", X_train.isnull().values.any())


¿Hay NaNs en X_train? False


In [ ]:
# Paso 6: Comparación de modelos alternativos
import time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Diccionario con modelos alternativos
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Lista para resultados
resultados = []

# Entrenamiento y evaluación
for nombre, modelo in modelos.items():
    inicio = time.time()
    modelo.fit(X_train, y_train)
    fin = time.time()

    y_pred = modelo.predict(X_test)

    resultados.append({
        "Modelo": nombre,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='macro'),
        "Recall": recall_score(y_test, y_pred, average='macro'),
        "F1 Score": f1_score(y_test, y_pred, average='macro'),
        "Tiempo entrenamiento (s)": round(fin - inicio, 4)
    })

# Mostrar resultados en DataFrame
df_resultados = pd.DataFrame(resultados).sort_values(by="F1 Score", ascending=False)
df_resultados


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Modelo,Accuracy,Precision,Recall,F1 Score,Tiempo entrenamiento (s)
5,Gradient Boosting,0.971429,0.958333,0.958333,0.955556,0.3576
3,Decision Tree,0.914286,0.900000,0.875000,0.864253,0.0020
1,Random Forest,0.885714,0.884127,0.815476,0.826887,0.1978
0,Logistic Regression,0.742857,0.592593,0.595238,0.592760,0.2815
4,K-Nearest Neighbors,0.371429,0.265152,0.241667,0.246032,0.0022
2,SVM,0.571429,0.190476,0.333333,0.242424,0.0026


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Definir la malla de hiperparámetros
param_grid_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gb = GradientBoostingClassifier(random_state=42)
grid_gb = GridSearchCV(gb, param_grid_gb, cv=5, scoring='f1', n_jobs=-1)
grid_gb.fit(X_train, y_train)

print("Mejores parámetros (Gradient Boosting):", grid_gb.best_params_)


Mejores parámetros (Gradient Boosting): {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definir malla de hiperparámetros
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf,
                       cv=5, scoring='f1_macro', n_jobs=-1)

grid_rf.fit(X_train, y_train)

print("✅ Mejores hiperparámetros (Random Forest):", grid_rf.best_params_)


✅ Mejores hiperparámetros (Random Forest): {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Redefinir modelos con hiperparámetros óptimos
mejor_gb = GradientBoostingClassifier(
    learning_rate=0.01, max_depth=3, n_estimators=50, random_state=42
)

mejor_rf = RandomForestClassifier(
    max_depth=10, min_samples_split=5, n_estimators=50, random_state=42
)

modelos_ajustados = {
    "Gradient Boosting Ajustado": mejor_gb,
    "Random Forest Ajustado": mejor_rf
}

# Evaluar modelos ajustados
resultados_ajustados = []

for nombre, modelo in modelos_ajustados.items():
    inicio = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    fin = time.time()

    resultados_ajustados.append({
        "Modelo": nombre,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average="macro", zero_division=0),
        "Recall": recall_score(y_test, y_pred, average="macro"),
        "F1 Score": f1_score(y_test, y_pred, average="macro"),
        "Tiempo entrenamiento (s)": round(fin - inicio, 4)
    })

# Mostrar resultados en un DataFrame
import pandas as pd
df_resultados_ajustados = pd.DataFrame(resultados_ajustados)
df_resultados_ajustados


,Modelo,Accuracy,Precision,Recall,F1 Score,Tiempo entrenamiento (s)
0,Gradient Boosting Ajustado,0.771429,0.697262,0.636905,0.639489,0.5452
1,Random Forest Ajustado,0.885714,0.845238,0.821429,0.831614,0.2179


### Conclusión

Tras comparar los modelos ajustados, seleccionamos **Random Forest Ajustado** como el modelo final, ya que obtuvo el mejor desempeño general:
- **F1 Score**: 0.8316
- **Accuracy**: 0.8857
- **Tiempo de entrenamiento**: 0.2179 segundos

Además, logró mayor balance entre precisión y recall, por lo que representa la mejor opción para resolver nuestro problema de clasificación.
